# Langchain for Conversational Memory

In [ ]:
import os
from google import genai
from langchain_google_genai import ChatGoogleGenerativeAI


# 1. Setup your key
os.environ["GEMINI_API_KEY"] = "XXX"
# 2. Initialize the model 
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
# 3. Invoke the model
result = llm.invoke("What is 1+1")
print(result.content)


1 + 1 = 2


## How RunnableWithMessageHistory Works

**User input:** `{"input": "Hi"}`

↓

**Step 1:** `get_session_history("user123")` retrieves conversation history
```
[Message1, Message2, ...]
```

↓

**Step 2:** LangChain automatically creates a combined dictionary
```python
{
    "history": [Message1, Message2, ...],
    "input": "Hi"
}
```

↓

**Step 3:** Prompt template receives this dict and fills in:
- `MessagesPlaceholder(variable_name="history")` ← gets `[Message1, Message2, ...]`
- `{input}` ← gets `"Hi"`

↓

**Result:** Complete prompt is sent to the LLM with full conversation context


In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.tracers.stdout import ConsoleCallbackHandler  # Used for verbose

set_verbose(True)


# Create a prompt template with message history placeholder
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant."),
    MessagesPlaceholder(variable_name="chat_history"),       # Messages is stored in this variable "chat_history". Variable name is self-defined.
    ("human", "{input}")
])

# Create the chain using LCEL (pipe operator)
chain = prompt | llm                                        # Creates a sequence to connect prompt template to language model.

# Set up session storage
store = {}

def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()    # InMemoryChatMessageHistory stores messages in RAM
    return store[session_id]

# Wrap the chain with message history
chain_with_history = RunnableWithMessageHistory(            # Manages message flow by retrieving history, load past messages, insert into prompt
    chain,
    get_session_history,
    input_messages_key="input",                             # Input is where the user's message is stored in
    history_messages_key="chat_history"                     # {"chat_history": [past_messages], "input": "current message"}
)

# Use it
response = chain_with_history.invoke(
    {"input": "Hi, I'm Eugene, a Data Scientist, and I work in a FAANG company."},
    config={"configurable": {"session_id": "user123"}}
)


# Follow-up message (history is automatically maintained)
response = chain_with_history.invoke(
    {"input": "What's my name?"},
    config={"configurable": {"session_id": "user123"}}
)

print(response.content)

# Follow-up message (history is automatically maintained)
response = chain_with_history.invoke(
    {"input": "What is my job?"},
    config={"configurable": {"session_id": "user123"}}
)

print(response.content)


response = chain_with_history.invoke(
    {"input": "Which company does he work in?"},
    config={
        "configurable": {"session_id": "user123"},
        "callbacks": [ConsoleCallbackHandler()]  # This enables verbose output
    }
)




Your name is Eugene. You mentioned it in your first message.
You are a Data Scientist working in a FAANG company.
[chain/start] [chain:RunnableWithMessageHistory] Entering Chain run with input:
{
  "input": "Which company does he work in?"
}
[chain/start] [chain:RunnableWithMessageHistory > chain:insert_history] Entering Chain run with input:
{
  "input": "Which company does he work in?"
}
[chain/start] [chain:RunnableWithMessageHistory > chain:insert_history > chain:RunnableParallel<chat_history>] Entering Chain run with input:
{
  "input": "Which company does he work in?"
}
[chain/start] [chain:RunnableWithMessageHistory > chain:insert_history > chain:RunnableParallel<chat_history> > chain:load_history] Entering Chain run with input:
{
  "input": "Which company does he work in?"
}
[chain/end] [chain:RunnableWithMessageHistory > chain:insert_history > chain:RunnableParallel<chat_history> > chain:load_history] s] Exiting Chain run with output:
[outputs]
[chain/end] [chain:RunnableWithM